In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
os.chdir('/Users/colerichmond/Documents/GitHub/DSC180B-Capstone')
!pwd

/Users/colerichmond/Documents/GitHub/DSC180B-Capstone


In [14]:
!python run.py data

Ingesting data...
...done!
Cleaning data...
...done!


In [3]:
sc_cols = ["county_name",
             "subject_age"]
fl_cols = [
      "county_name",
      "officer_sex",
      "officer_years_of_service",
      "subject_age",
      "time",
      "vehicle_registration_state"
   ]
pa_cols = [
      "time",
      "lat",
      "lng",
      "officer_sex",
      "contraband_found"
   ]

In [4]:
all_cols = [
      "subject_race",
      "subject_sex",
      "officer_race",
      "officer_age",
      "violation",
      "arrest_made",
      "citation_issued",
      "search_conducted"
   ]

In [8]:
def get_stops_url(location, all_cols, sc_cols, fl_cols, pa_cols):
    '''
    return a downloadable file of traffic
    stops for a given year before RIPA.
    
    :param: given year to fetch
    '''
    
    if not os.path.exists('data/raw'):
        os.makedirs('data/raw')
        
    if location is "pa":
        url = 'https://stacks.stanford.edu/file/druid:yg821jf8611/yg821jf8611_pa_pittsburgh_2020_04_01.csv.zip'
        cols = all_cols + pa_cols
        table = pd.read_csv(url, nrows=100).loc[:, cols].dropna()
    else:
        url = 'https://stacks.stanford.edu/file/druid:kx738rc7407/kx738rc7407_%s_statewide_2019_12_17.csv.zip' % (location)
        if location is "sc":  
            cols = all_cols + sc_cols
            table = pd.read_csv(url, nrows=100).loc[:, cols]
            cols.remove("violation")
            table = table.dropna(subset=cols)
        if location is "fl":
            cols = all_cols + fl_cols
            table = pd.read_csv(url, nrows=100).loc[:, cols].dropna()

    return table

In [9]:
for location in ["pa", "fl", "sc"]:
    table = get_stops_url(location, all_cols, sc_cols, fl_cols, pa_cols)
    file_name = '%s_stops.csv' % (location)
    table.to_csv(os.path.join("data/raw", file_name))

In [10]:
!pwd

/Users/colerichmond/Documents/GitHub/DSC180B-Capstone


In [11]:
pd.read_csv('data/raw/sc_stops.csv')

,Unnamed: 0,subject_race,subject_sex,officer_race,officer_age,violation,arrest_made,citation_issued,search_conducted,county_name,subject_age
0,0,white,male,white,28.0,56-10-225 Other Violations,False,True,False,Georgetown County,33
1,1,white,male,white,34.0,NaN,False,False,False,Anderson County,17
2,2,white,male,white,44.0,56-5-1520 Speeding or Too Fast For Conditions ...,False,True,False,Greenville County,68
3,3,black,female,white,31.0,56-5-1520 Speeding or Too Fast For Conditions ...,False,True,False,Spartanburg County,27
4,4,white,male,black,32.0,56-5-1520 Speeding or Too Fast For Conditions ...,False,True,False,Darlington County,29
...,...,...,...,...,...,...,...,...,...,...,...
94,95,white,male,white,49.0,56-5-1520 Speeding or Too Fast For Conditions ...,False,True,False,Spartanburg County,36
95,96,white,female,white,30.0,56-1-460 Driving Under Suspension,False,True,False,Charleston County,26
96,97,white,male,black,28.0,NaN,False,False,False,Orangeburg County,29
97,98,white,female,white,38.0,NaN,False,False,False,Marion County,29


In [ ]:
cols = all_cols + sc_cols
cols.remove("violation")
cols
table = table.dropna(subset=cols)
table

In [12]:
os.listdir('data/raw')

['pa_stops.csv', 'fl_stops.csv', 'sc_stops.csv']